In [3]:
import pandas as pd
import psycopg2 as postgres


In [4]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

def connect():
    conn = postgres.connect(
        host='localhost',
        database=BASE,
        user=USER,
        password=PSWD,
        port='5432'
    )
    return conn
conn = connect()
cur = conn.cursor()

In [553]:
cur.execute(open("sql/questions/reset.sql", "r").read())

In [554]:
# requête interactive
region = 'aquitaine' # mettre a jour le contenu ici pour changer de région (i.e: tagne -> Bretagne, compte -> Franche compté etc)

cur.execute("select d.libelle from departements d join regions r using(reg) where r.ncc LIKE UPPER(%(region)s) or r.libelle LIKE UPPER(%(region)s) order by d.libelle", {'region': f'%{region}%'})
conn.commit()
data = cur.fetchall()

print(f'Les départements qui correspondent a la région {region}:')
for dep in data:
    print(f'\t- {dep[0]}')


Les départements qui correspondent a la région aquitaine:
	- Charente
	- Charente-Maritime
	- Corrèze
	- Creuse
	- Deux-Sèvres
	- Dordogne
	- Gironde
	- Haute-Vienne
	- Landes
	- Lot-et-Garonne
	- Pyrénées-Atlantiques
	- Vienne


In [555]:
commune = 'bordeaux'  # le nom de la ville doit être exact, sans accent ni caractère spéciaux
indicateur = 'pop'  # parmis DECE, LOG, LOGVAC, NAIS, NPER, PMEN, POP, RP, RSECOCC, SUPERF

cur.execute("select annee, ncc, dep,valeur from communes c join statistiques s on s.codgeo = c.com where upper(s.indicateur) = upper(%(indicateur)s) and c.ncc = upper( %(commune)s)", 
            {'commune': commune, 'indicateur': indicateur})
conn.commit()
data = cur.fetchall()

if(len(data)):
    print(f"Information pour l'indicateur {indicateur} pour la commune {commune} ({data[0][2]})")
    for info in data:
        print(f'\t{info[0]}: {info[3]:,}')
else:
    print(f'aucune information pour la commune de {commune} et l\'indicateur {indicateur}')

Information pour l'indicateur pop pour la commune bordeaux (33 )
	1968: 257,070
	1975: 257,070
	1982: 257,070
	1990: 257,070
	1999: 257,070
	2008: 257,070
	2013: 257,070
	2018: 257,871


In [556]:
# Question 1
cur.execute(open("sql/questions/question_1/question_1#1.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Les 10 villes les plus peuplées de France en 2018:')
for ville in data:
    print(f'\t- {ville[0]}: {ville[1]:,} habitants')


Les 10 villes les plus peuplées de France en 2018:
	- Paris: 2,175,702 habitants
	- Marseille: 868,277 habitants
	- Lyon: 518,635 habitants
	- Toulouse: 486,828 habitants
	- Nice: 341,032 habitants
	- Montpellier: 290,053 habitants
	- Strasbourg: 284,677 habitants
	- Bordeaux: 257,871 habitants
	- Lille: 233,098 habitants
	- Rennes: 217,728 habitants


In [557]:
# Question 2
cur.execute(open("sql/questions/question_1/question_1#2.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Évolution de la population Française au fil des années:')
for population in data:
    population = [0 if v is None else v for v in population]
    print(f'\n\t- Année {population[0]}: {population[1]:,} habitants -> +{population[2]:,} habitants ({population[3]}%)')
    print(f'\t\t- Naissances: {population[4]:,} | Décès: {population[5]:,}')
    if(population[2]):
        print(f'\t\t- Augmentation: {population[2]:,} || Naissances - Décès: {population[4] - population[5]:,} || Différence: {population[2]-(population[4] - population[5]):,}')


Évolution de la population Française au fil des années:

	- Année 1968: 50,788,520 habitants -> +0 habitants (0%)
		- Naissances: 6,378,063 | Décès: 4,144,525

	- Année 1975: 53,798,003 habitants -> +3,009,483 habitants (5.5940422175150255%)
		- Naissances: 5,767,687 | Décès: 4,149,276
		- Augmentation: 3,009,483 || Naissances - Décès: 1,618,411 || Différence: 1,391,072

	- Année 1982: 55,618,453 habitants -> +1,820,450 habitants (3.273104341826983%)
		- Naissances: 6,599,046 | Décès: 4,613,309
		- Augmentation: 1,820,450 || Naissances - Décès: 1,985,737 || Différence: -165,287

	- Année 1990: 58,087,393 habitants -> +2,468,940 habitants (4.250388720320087%)
		- Naissances: 7,110,433 | Décès: 5,057,784
		- Augmentation: 2,468,940 || Naissances - Décès: 2,052,649 || Différence: 416,291

	- Année 1999: 60,191,608 habitants -> +2,104,215 habitants (3.495861084156449%)
		- Naissances: 7,442,930 | Décès: 5,074,193
		- Augmentation: 2,104,215 || Naissances - Décès: 2,368,737 || Différence: -

In [558]:
# Question 3
cur.execute(open("sql/questions/question_1/question_1#3.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Ville la plus peuplée de chaque région en 2018:')
for population in data:
    print(f'\t- Région {population[2]} -> {population[1]} ({population[0]}): {population[3]:,} habitants')


Ville la plus peuplée de chaque région en 2018:
	- Région Île-de-France -> Paris (75056): 2,175,702 habitants
	- Région Provence-Alpes-Côte d'Azur -> Marseille (13055): 868,277 habitants
	- Région Auvergne-Rhône-Alpes -> Lyon (69123): 518,635 habitants
	- Région Occitanie -> Toulouse (31555): 486,828 habitants
	- Région Grand Est -> Strasbourg (67482): 284,677 habitants
	- Région Nouvelle-Aquitaine -> Bordeaux (33063): 257,871 habitants
	- Région Hauts-de-France -> Lille (59350): 233,098 habitants
	- Région Bretagne -> Rennes (35238): 217,728 habitants
	- Région Normandie -> Le Havre (76351): 169,733 habitants
	- Région Bourgogne-Franche-Comté -> Dijon (21231): 156,854 habitants
	- Région Pays de la Loire -> Angers (49007): 154,508 habitants
	- Région La Réunion -> Saint-Denis (97411): 150,535 habitants
	- Région Centre-Val de Loire -> Tours (37261): 136,463 habitants
	- Région Martinique -> Fort-de-France (97209): 78,126 habitants
	- Région Corse -> Ajaccio (2A004): 70,817 habitants
	

In [559]:
# Création des vues

cur.execute(open("sql/questions/question_2/question_2#vue_region.sql", "r").read())
cur.execute(open("sql/questions/question_2/question_2#vue_departements.sql", "r").read())
conn.commit()


In [560]:
# Ajout des colonnes population
cur.execute(open("sql/questions/question_3/alter_tables.sql", "r").read())
cur.execute("select ncc,population from regions order by ncc limit 5;")
conn.commit()

data = cur.fetchall()
for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: None
BOURGOGNE FRANCHE COMTE: None
BRETAGNE: None
CENTRE VAL DE LOIRE: None
CORSE: None


In [561]:

# Creation et appel de la procédure
cur.execute(open("sql/questions/question_3/procedure.sql", "r").read())
cur.execute("call proc_update_population()")
cur.execute("select ncc,population from regions order by ncc limit 5;")
conn.commit()

data = cur.fetchall()
for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: 7994459
BOURGOGNE FRANCHE COMTE: 2807807
BRETAGNE: 3335226
CENTRE VAL DE LOIRE: 2572853
CORSE: 338554


# Note
Les 2 blocs suivants sont executés dans l'ordre inverse de leur appartition sur l'ennoncé du TP, nous n'avons pas réussi a rendre opérationnel la mise a jour automatiques des populations si les tables régions et départements rejettent toute mise a jour/insertion/suppression.

On aurait pu bloquer les tables au niveau des droits et permettre a la procédure de s'executer mais la question portant sur les trigger nous avons fait le blocage par trigger.

Ainsi pour tester elles s'executent dans l'ordre inverse à la demande:
1. création d'un trigger sur la table de statistiques qui ne se déclenche qu'à la suppression ou si la mise à jour/insertion implique l'indicateur de population
2. mise a jour de la tables statistiques pour augmenter une population (cf: cellule suivante)
3. création des triggers de blocage d'opération
4. test de mise a jour des tables régions et départements afin de constater que l'erreur que nous renvoyons depuis le trigger nous ai bien retournée

In [562]:
# Mise a jour auto des tables régions et départements quand un indicateur de population est modifié

ville = 'bordeaux'  # ville ciblée (se base sur le ncc)
augmentation = 1 # permet de définir de combien l'on augmente la population de la ville donnée

# ne pas modifier en dessous
sql = """
    select r.libelle,r.population, d.libelle,d.population, c.libelle,valeur::integer, c.com
    from communes c 
    join statistiques s on s.codgeo = c.com 
    join departements d using(dep) 
    join regions r using(reg) 
    where c.ncc = upper(%(ville)s) 
    and s.annee = 2018 
    and s.indicateur = 'POP'
    group by valeur, r.population, d.population, r.libelle, c.libelle, d.libelle, c.com;
""" # ne pas modifier


# dictionnaires utilisés pour constater les différences
pre_update = {'region' : None, 'dep': None, 'ville': None} 
post_update = {'region': None, 'dep': None, 'ville': None}

# ajout du trigger
cur.execute(open("sql/questions/question_3/triggers_2.sql", "r").read())
cur.execute(sql, {'ville': ville})
conn.commit()
pre_update = list(cur.fetchone())
try:
    cur.execute("update statistiques set valeur = %(valeur)s where indicateur = 'POP' and annee = 2018 and codgeo = %(codgeo)s;", {'valeur': pre_update[-2]+augmentation, 'codgeo': str(pre_update[-1])})
    cur.execute(sql, {'ville': ville})
    conn.commit()
    post_update = list(cur.fetchone())
    print("Avant mise à jour:")
    print(f'Région: {pre_update[0]} -> {pre_update[1]} habitants')
    print(f'Département: {pre_update[2]} -> {pre_update[3]} habitants')
    print(f'Ville: {pre_update[4]} -> {pre_update[5]} habitants')
    print("\nAprès mise à jour:")
    print(f'Région: {post_update[0]} -> {post_update[1]} habitants')
    print(f'Département: {post_update[2]} -> {post_update[3]} habitants')
    print(f'Ville: {post_update[4]} -> {post_update[5]} habitants')
except Exception as error:
    print(error)


Avant mise à jour:
Région: Nouvelle-Aquitaine -> 5980581 habitants
Département: Gironde -> 1602648 habitants
Ville: Bordeaux -> 257871 habitants

Après mise à jour:
Région: Nouvelle-Aquitaine -> 5980582 habitants
Département: Gironde -> 1602649 habitants
Ville: Bordeaux -> 257872 habitants


In [563]:
# Blocage des tables régions et départements

cur.execute(open("sql/questions/question_3/triggers_1.sql", "r").read())
conn.commit()
try:
    cur.execute("update regions set population = 10000 where reg = '75';")
except Exception as error:
    print(error)  # UPDATE on regions is not allowed.
try:
    conn = connect()
    cur = conn.cursor()
    cur.execute("update departements set population = 10000 where dep = '33';")
except Exception as error:
    print(error)  # UPDATE on departements is not allowed.


UPDATE on regions is not allowed.
CONTEXT:  PL/pgSQL function fun_prevent() line 3 at RAISE

UPDATE on departements is not allowed.
CONTEXT:  PL/pgSQL function fun_prevent() line 3 at RAISE



In [591]:
# Plans d'executions

#1
print('#1 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18)\n')
cur.execute(open("sql/questions/question_4/explain_small_to_small.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#1 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18)

Hash Join  (cost=2.41..28.45 rows=18 width=87)
  Hash Cond: ((regions_cheflieu.reg)::text = (regions.reg)::text)
  ->  Seq Scan on regions_cheflieu  (cost=0.00..22.70 rows=1270 width=36)
  ->  Hash  (cost=2.18..2.18 rows=18 width=33)
        ->  Seq Scan on regions  (cost=0.00..2.18 rows=18 width=33)


In [585]:
#2
print('#2 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361)\n')
cur.execute(
    open("sql/questions/question_4/explain_smallest_to_biggest.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#2 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361)

Hash Join  (cost=13.15..46904.81 rows=2446515 width=574)
  Hash Cond: ((statistiques.indicateur)::text = (indicateurs.code)::text)
  ->  Seq Scan on statistiques  (cost=0.00..40275.15 rows=2446515 width=20)
  ->  Hash  (cost=11.40..11.40 rows=140 width=554)
        ->  Seq Scan on indicateurs  (cost=0.00..11.40 rows=140 width=554)


In [5]:

#3
print('#3 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18) avec tri et ordonnancement\n')
cur.execute(
    open("sql/questions/question_4/explain_small_to_small_ordered.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])

#3 - Jointure entre les 2 plus petites tables regions et regions_cheflieu card(18:18) avec tri et ordonnancement

Sort  (cost=10.42..10.42 rows=1 width=87)
  Sort Key: ((regions.reg)::character varying)
  ->  Nested Loop  (cost=0.15..10.41 rows=1 width=87)
        ->  Seq Scan on regions  (cost=0.00..2.23 rows=1 width=33)
              Filter: ((ncc)::text ~~ '%BRETA%'::text)
        ->  Index Scan using regions_cheflieu_pkey on regions_cheflieu  (cost=0.15..8.17 rows=1 width=36)
              Index Cond: ((reg)::text = (regions.reg)::text)


In [589]:
#4
print('#4 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361) avec tri et ordonnancement\n')
cur.execute(open(
    "sql/questions/question_4/explain_smallest_to_biggest_ordered.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])

#4 - Jointure entre la plus petite table (indicateurs) et la plus grande (statistiques) card(10:2446361) avec tri et ordonnancement

Nested Loop  (cost=37422.54..38601.06 rows=9076 width=574)
  ->  Gather Merge  (cost=37422.39..38479.44 rows=9076 width=20)
        Workers Planned: 2
        ->  Sort  (cost=36422.37..36431.82 rows=3782 width=20)
              Sort Key: statistiques.valeur
              ->  Parallel Seq Scan on statistiques  (cost=0.00..36197.62 rows=3782 width=20)
                    Filter: (((indicateur)::text = 'POP'::text) AND (annee = 2018) AND ((valeur)::integer > 100000))
  ->  Materialize  (cost=0.14..8.17 rows=1 width=554)
        ->  Index Scan using indicateurs_pkey on indicateurs  (cost=0.14..8.16 rows=1 width=554)
              Index Cond: ((code)::text = 'POP'::text)


## Constats pour les 2eme et 4eme plans d'execution

Par rapport a son homologue (#2) on constate une différence de coût pour la requête (#4), le coût final de cette requête est bien plus petit que celui de la requête qui ne faisait que la jointure (46904.81 contre 38601.06) pour un nombre de ligne très largement inferieur dans cette requête (2446515 contre 9076).

On peut expliquer la différence de coût par le fait que les 2 requêtes ont été executée a la suite lors du test, la première aura peuplé le cache de la base en aillant déjà fait les calculs nécéssaires sur les index de la jointure entre indicateurs et statisitques. La numéro 4 n'étant qu'une extension de la requête 2 (on a juste ajouté des filtres et tri)

On note une grosse différence c'est l'apparition du Workers Planned, qui nous informe qu'en vu d'optimiser le temps de calcul le plannificateur prévois d'utiliser des processus auxiliaires pour executer l'ensemble de la requête (ici 2 processus), s'il intervient ici et n'était pas apparu lors de la #1 c'est certainement a cause du filtre comme on le voit avec le scan parallèle (job auxiliaire) qui applique le filtre.

En général la 2eme requête est bien plus optimisée que celle qui fait uniquement la jointure.


## Comparaison des duo (2-4) et (1-3)
les requêtes #1 et #3 étants effectuées sur de très petites tables (18 entrées) on constate que le plannificateur a tout de même optimisé le temps de calcul pour la #3 qui applique des filtres sur la requête.
La où c'est interessants c'est en comparant la #3 et la #4 qui sont similaire (jointure + filtre) le planificateur n'a pas fait appel a des processus auxiliaires contrairement a la #4, probablement a cause de la taille des tables le plannificateur doit estimer que l'utilisation d'un processus auxiliaire ne ferait pas gagner de temps dans ce cas là.

In [6]:
#5
print('#5 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965)\n')
cur.execute(open("sql/questions/question_4/explain_triple_join.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#5 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965)

Hash Join  (cost=20.07..1137.49 rows=34965 width=150)
  Hash Cond: (communes.dep = (departements.dep)::bpchar)
  ->  Seq Scan on communes  (cost=0.00..636.65 rows=34965 width=35)
  ->  Hash  (cost=18.80..18.80 rows=101 width=60)
        ->  Hash Join  (cost=2.41..18.80 rows=101 width=60)
              Hash Cond: (departements.reg = (regions.reg)::bpchar)
              ->  Seq Scan on departements  (cost=0.00..15.01 rows=101 width=30)
              ->  Hash  (cost=2.18..2.18 rows=18 width=33)
                    ->  Seq Scan on regions  (cost=0.00..2.18 rows=18 width=33)


In [7]:
#6
print('#6 - Multiples jointures entre les tables regions, departements et communes card(18:101:34965) mais la jointure est "inversée"\n')
cur.execute(open("sql/questions/question_4/explain_triple_join_revert.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#6 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965) mais la jointure est "inversée"

Hash Join  (cost=20.07..1137.49 rows=34965 width=150)
  Hash Cond: (communes.dep = (departements.dep)::bpchar)
  ->  Seq Scan on communes  (cost=0.00..636.65 rows=34965 width=35)
  ->  Hash  (cost=18.80..18.80 rows=101 width=61)
        ->  Hash Join  (cost=2.41..18.80 rows=101 width=61)
              Hash Cond: (departements.reg = (regions.reg)::bpchar)
              ->  Seq Scan on departements  (cost=0.00..15.01 rows=101 width=30)
              ->  Hash  (cost=2.18..2.18 rows=18 width=33)
                    ->  Seq Scan on regions  (cost=0.00..2.18 rows=18 width=33)


## Constat entre la 5 et la 6
La 5 part de régions et joint départements puis communes alors que la 6 part de communes et fini sur régions
On constate que le plannificateur effectue exactements les mêmes requêtes peut importe l'ordre d'appartition des tables dans le select et ses jointures

In [8]:
#7
print('#7 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M)\n')
cur.execute(open("sql/questions/question_4/explain_quad_join.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#7 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M)

Hash Join  (cost=1093.78..81431.39 rows=2446515 width=170)
  Hash Cond: (communes.dep = (departements.dep)::bpchar)
  ->  Hash Join  (cost=1073.71..47771.74 rows=2446515 width=55)
        Hash Cond: ((statistiques.codgeo)::text = (communes.com)::text)
        ->  Seq Scan on statistiques  (cost=0.00..40275.15 rows=2446515 width=20)
        ->  Hash  (cost=636.65..636.65 rows=34965 width=35)
              ->  Seq Scan on communes  (cost=0.00..636.65 rows=34965 width=35)
  ->  Hash  (cost=18.80..18.80 rows=101 width=61)
        ->  Hash Join  (cost=2.41..18.80 rows=101 width=61)
              Hash Cond: (departements.reg = (regions.reg)::bpchar)
              ->  Seq Scan on departements  (cost=0.00..15.01 rows=101 width=30)
              ->  Hash  (cost=2.18..2.18 rows=18 width=33)
                    ->  Seq Scan on regions  (cost=0.00..2.18 rows=18 width=33)


In [12]:
#8
print('#8 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri et ordonancement\n')
cur.execute(open("sql/questions/question_4/explain_quad_join_ordered.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#8 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri et ordonancement

Gather Merge  (cost=49496.46..73817.79 rows=208454 width=170)
  Workers Planned: 2
  ->  Sort  (cost=48496.44..48757.01 rows=104227 width=170)
        Sort Key: statistiques.annee
        ->  Hash Join  (cost=788.82..30901.95 rows=104227 width=170)
              Hash Cond: (communes.dep = (departements.dep)::bpchar)
              ->  Hash Join  (cost=768.75..29448.76 rows=104227 width=55)
                    Hash Cond: ((statistiques.codgeo)::text = (communes.com)::text)
                    ->  Parallel Seq Scan on statistiques  (cost=0.00..26003.81 rows=1019381 width=20)
                    ->  Hash  (cost=724.06..724.06 rows=3575 width=35)
                          ->  Seq Scan on communes  (cost=0.00..724.06 rows=3575 width=35)
                                Filter: ((ncc)::text ~~ 'B%'::text)
              ->  Hash  (cost=18.80..18.80 rows=101

# Constats entre (7-8) et (4-8)
## 7-8
Comme pour 2-4 on constate que la requête avec des tri est moins coûteuse que la jointure seule également tout ce qui est filtre et ordernancement est géré par des processus auxiliaires (2 également, on peut se demander si ce n'est pas un pour les filtres et un pour le order by, réponse dans la cellule suivante)

## 4-8
Les 2 requêtes sont similaire (le coût dans la branche des workers est très similaire ~30k) et pourtant la 8 consomme plus que la 4, les jointures impactent sont très certainement à mettre en cause bien que d'autres facteurs entrent en jeux (le fait que toutes soient faites a la suite, la machine etc)

In [13]:
#9
print('#9 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri\n')
cur.execute(open("sql/questions/question_4/explain_quad_join_filtered.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#9 - Multiples jointures entre les tables regions, departements, communes et statistiques card(18:101:34965:~2M) avec tri

Hash Join  (cost=788.82..50926.33 rows=250144 width=170)
  Hash Cond: (communes.dep = (departements.dep)::bpchar)
  ->  Hash Join  (cost=768.75..47466.78 rows=250144 width=55)
        Hash Cond: ((statistiques.codgeo)::text = (communes.com)::text)
        ->  Seq Scan on statistiques  (cost=0.00..40275.15 rows=2446515 width=20)
        ->  Hash  (cost=724.06..724.06 rows=3575 width=35)
              ->  Seq Scan on communes  (cost=0.00..724.06 rows=3575 width=35)
                    Filter: ((ncc)::text ~~ 'B%'::text)
  ->  Hash  (cost=18.80..18.80 rows=101 width=61)
        ->  Hash Join  (cost=2.41..18.80 rows=101 width=61)
              Hash Cond: (departements.reg = (regions.reg)::bpchar)
              ->  Seq Scan on departements  (cost=0.00..15.01 rows=101 width=30)
              ->  Hash  (cost=2.18..2.18 rows=18 width=33)
                    ->  Seq Scan on

## constat entre 8 et 9
La seule différence entre les 2 est le order by qui a été enlevé dans la 9, on constate que le plannificateur ne fait plus appel aux processus auxiliaires ce qui peut confirmer l'hypothèse émise précedement

In [5]:
#10
print('#10 - Multiples jointures et fonction d\'aggrégat\n')
cur.execute(open("sql/questions/question_4/explain_agregate.sql", "r").read())
conn.commit()
for state in cur.fetchall():
    print(state[0])


#10 - Multiples jointures et fonction d'aggrégat

Sort  (cost=71486.20..71486.20 rows=2 width=69)
  Sort Key: groupe_region_population.valeur_max DESC
  ->  Nested Loop  (cost=37535.70..71486.19 rows=2 width=69)
        ->  Index Only Scan using indicateurs_pkey on indicateurs i  (cost=0.14..8.16 rows=1 width=38)
              Index Cond: (code = 'POP'::text)
        ->  Hash Join  (cost=37535.55..71478.00 rows=2 width=70)
              Hash Cond: (d.reg = (r.reg)::bpchar)
              ->  Hash Join  (cost=37533.15..71475.57 rows=2 width=62)
                    Hash Cond: ((c.dep = (d.dep)::bpchar) AND (groupe_region_population.reg = d.reg))
                    ->  Nested Loop  (cost=37516.62..71458.68 rows=45 width=63)
                          ->  Hash Join  (cost=37516.33..71442.30 rows=45 width=46)
                                Hash Cond: (s.valeur = groupe_region_population.valeur_max)
                                ->  Gather  (cost=1000.00..34823.42 rows=27227 width=17)
    

# constat pour la 10
Le plan étant conséquent nous nous somme servi d'un outil pour analyser les plans d'execution https://explain.dalibo.com/plan/6T4
Dans l'ensemble on constate que le planificateur fait appel a tout ce que l'on a vu précédement, de multiples fois en raison de la sous requête.
L'apparition du groupe HashAggregate est nouveau, il intervient a cause de la fonction d'aggrégat utilisée dans la requête. On constate en revanche que malgré la "complexité" de cette requête son coup est moins elevé que la n°8 alors qu'elle effectue des jointures similaires.

In [588]:
# conn.close()